In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import re
#to access our implemented ML functions
%load_ext autoreload
%autoreload 2


In [116]:
from implementations import *

In [3]:
Energy_range = [0.1, 0.2, 0.3, 0.5, 1.0, 2.0, 3.0, 5.0, 10.0, 20.0]
data_path = data_file(0.1)

In [4]:
df = build_df(data_path)

In [5]:
df[-17:-1]

,X(mm),Y(mm),Z(mm),DX,DY,DZ,KinE(MeV),dE(MeV),StepLeng,TrackLeng,NextVolume,ProcName
*,,,,,,,,,,,,
17,0.506,99.8,99.9,0.0842,0.354,-0.931,0.0565,0.00529,0.0431,0.783,phantom,msc
18,0.52,99.9,99.8,0.175,0.684,-0.708,0.0463,0.0102,0.229,1.01,phantom,eIoni
19,0.544,100,99.7,0.435,0.737,0.518,0.0263,0.02,0.195,1.21,phantom,eIoni
20,0.569,100,99.7,0.313,0.949,-0.0254,0.00728,0.019,0.111,1.32,phantom,eIoni
21,0.571,100,99.7,0.313,0.949,-0.0254,0,0.00728,0.0118,1.33,phantom,eIoni
End,of,Run,Action:,Dimensions,1,1,1,voxel,volume,20000000,NaN,NaN
opened,file,guava.out,for,dose,output,0,phantomSD,NaN,NaN,NaN,NaN,NaN
TOTAL,40000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Finished,:,End,of,Run,Action,0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.head()

,X(mm),Y(mm),Z(mm),DX,DY,DZ,KinE(MeV),dE(MeV),StepLeng,TrackLeng,NextVolume,ProcName
*,,,,,,,,,,,,
Step#,X(mm),Y(mm),Z(mm),DX,DY,DZ,KinE(MeV),dE(MeV),StepLeng,TrackLeng,NextVolume,ProcName
0,0,100,100,0,0,0,0.1,0,0,0,phantom,msc
1,0.0471,100,100,0.995,-0.0748,0.0695,0.098,0.00196,0.048,0.048,phantom,msc
2,0.0941,100,100,0.975,0.196,0.101,0.0969,0.00111,0.0473,0.0954,phantom,msc
3,0.142,100,100,0.955,0.0481,0.293,0.0944,0.00257,0.0499,0.145,phantom,msc


## Building of clean dataframe

In [7]:
df_clean = clean_df(df)

In [22]:
df_clean.head(24)

,index,DX,DY,DZ,KinE(MeV),dE(MeV),StepLeng,ProcName,DX_s,DY_s,DZ_s,Kin(MeV)_s,name_s
0,0,0,0,0,0.1,0,0,msc,NaN,NaN,NaN,NaN,NaN
1,1,0.995,-0.0748,0.0695,0.098,0.00196,0.048,msc,NaN,NaN,NaN,NaN,NaN
2,2,0.975,0.196,0.101,0.0969,0.00111,0.0473,msc,NaN,NaN,NaN,NaN,NaN
3,3,0.955,0.0481,0.293,0.0944,0.00257,0.0499,msc,NaN,NaN,NaN,NaN,NaN
4,4,0.917,-0.171,0.361,0.0927,0.00163,0.0482,msc,NaN,NaN,NaN,NaN,NaN
5,5,0.876,-0.113,0.468,0.0911,0.00166,0.0429,msc,NaN,NaN,NaN,NaN,NaN
6,6,0.815,-0.146,0.561,0.0888,0.00227,0.0516,msc,NaN,NaN,NaN,NaN,NaN
7,7,0.512,-0.324,0.795,0.0874,0.00137,0.0462,msc,NaN,NaN,NaN,NaN,NaN
8,8,0.723,-0.619,0.308,0.0866,0.000812,0.0441,msc,NaN,NaN,NaN,NaN,NaN
9,9,0.717,-0.546,0.432,0.0849,0.00172,0.044,msc,NaN,NaN,NaN,NaN,NaN


In [9]:
df_clean[-10:-1]

,index,DX,DY,DZ,KinE(MeV),dE(MeV),StepLeng,ProcName,DX_s,DY_s,DZ_s,Kin(MeV)_s,name_s
397547,12,0.767,-0.57,0.295,0.0723,0.00149,0.0425,msc,NaN,NaN,NaN,NaN,NaN
397548,13,0.0845,0.749,-0.657,0.0699,0.00239,0.0446,msc,NaN,NaN,NaN,NaN,NaN
397549,14,0.124,0.539,-0.833,0.0678,0.00203,0.0454,msc,NaN,NaN,NaN,NaN,NaN
397550,15,0.0928,0.233,-0.968,0.0656,0.00226,0.0447,msc,NaN,NaN,NaN,NaN,NaN
397551,16,-0.101,0.0986,-0.99,0.0618,0.00376,0.0439,msc,NaN,NaN,NaN,NaN,NaN
397552,17,0.0842,0.354,-0.931,0.0565,0.00529,0.0431,msc,NaN,NaN,NaN,NaN,NaN
397553,18,0.175,0.684,-0.708,0.0463,0.0102,0.229,eIoni,NaN,NaN,NaN,NaN,NaN
397554,19,0.435,0.737,0.518,0.0263,0.02,0.195,eIoni,NaN,NaN,NaN,NaN,NaN
397555,20,0.313,0.949,-0.0254,0.00728,0.019,0.111,eIoni,NaN,NaN,NaN,NaN,NaN


### Is it always Spawn_in_step == 1 ?

In [10]:
for E in Energy_range:
    df_check = clean_df(build_df(data_file(E)))
    Spawn_in_step = df_check[df_check.index == ':-----']['DZ']
    Spawn_in_step = Spawn_in_step.str.replace('(','',regex=True).str.replace('Rest=','',regex=True).astype('int')
    print('For Energy ', E, ' we have : ',Spawn_in_step[Spawn_in_step != 1].sum(), ' interactions that produce more than 1 one particule')

For Energy  0.1  we have :  0  interactions that produce more than 1 one particule


FileNotFoundError: [Errno 2] No such file or directory: '../Data/E_0.2.data'

In [12]:
#Try to build the input and outputs corresponding to build a ML model

In [18]:
df_clean.head()

,index,DX,DY,DZ,KinE(MeV),dE(MeV),StepLeng,ProcName,DX_s,DY_s,DZ_s,Kin(MeV)_s,name_s
0,0,0,0,0,0.1,0,0,msc,NaN,NaN,NaN,NaN,NaN
1,1,0.995,-0.0748,0.0695,0.098,0.00196,0.048,msc,NaN,NaN,NaN,NaN,NaN
2,2,0.975,0.196,0.101,0.0969,0.00111,0.0473,msc,NaN,NaN,NaN,NaN,NaN
3,3,0.955,0.0481,0.293,0.0944,0.00257,0.0499,msc,NaN,NaN,NaN,NaN,NaN
4,4,0.917,-0.171,0.361,0.0927,0.00163,0.0482,msc,NaN,NaN,NaN,NaN,NaN


In [19]:
np_data = df_clean.to_numpy()

In [21]:
np_data[0:5, 1]

array(['0', '0.995', '0.975', '0.955', '0.917'], dtype=object)

In [16]:
np_data.shape[0]

397557

In [49]:
data_int = []
for idx, data_x in enumerate(np_data[0:20, 0:7]) :
    print(data_x.astype(float))
np_data[0:20, 8]

[0.  0.  0.  0.  0.1 0.  0. ]
[ 1.       0.995   -0.0748   0.0695   0.098    0.00196  0.048  ]
[2.00e+00 9.75e-01 1.96e-01 1.01e-01 9.69e-02 1.11e-03 4.73e-02]
[3.00e+00 9.55e-01 4.81e-02 2.93e-01 9.44e-02 2.57e-03 4.99e-02]
[ 4.00e+00  9.17e-01 -1.71e-01  3.61e-01  9.27e-02  1.63e-03  4.82e-02]
[ 5.00e+00  8.76e-01 -1.13e-01  4.68e-01  9.11e-02  1.66e-03  4.29e-02]
[ 6.00e+00  8.15e-01 -1.46e-01  5.61e-01  8.88e-02  2.27e-03  5.16e-02]
[ 7.00e+00  5.12e-01 -3.24e-01  7.95e-01  8.74e-02  1.37e-03  4.62e-02]
[ 8.00e+00  7.23e-01 -6.19e-01  3.08e-01  8.66e-02  8.12e-04  4.41e-02]
[ 9.00e+00  7.17e-01 -5.46e-01  4.32e-01  8.49e-02  1.72e-03  4.40e-02]
[ 1.00e+01  5.94e-01 -5.62e-01  5.76e-01  8.29e-02  2.03e-03  4.38e-02]
[ 1.10e+01  6.77e-01 -5.31e-01  5.09e-01  8.12e-02  1.66e-03  4.26e-02]
[ 1.20e+01  8.81e-01 -4.26e-01  2.07e-01  7.85e-02  2.75e-03  4.49e-02]
[ 1.30e+01  6.57e-01 -6.88e-01  3.09e-01  7.67e-02  1.78e-03  4.17e-02]
[ 1.40e+01  6.00e-01 -7.44e-01  2.96e-01  7.35e-02  3.1

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan], dtype=object)

In [108]:
X, y = creation_array(df_clean)

In [109]:
type(X)

numpy.ndarray

In [118]:
X[0:5]

array([[0.       , 0.1      , 0.       , 0.       , 1.       ],
       [0.       , 0.098    , 0.00196  , 0.048    , 1.       ],
       [0.9624837, 0.0969   , 0.00111  , 0.0473   , 1.       ],
       [0.9701456, 0.0944   , 0.00257  , 0.0499   , 1.       ],
       [0.9732829, 0.0927   , 0.00163  , 0.0482   , 1.       ]])

In [122]:
y[:,1].shape

(377557,)

In [113]:
y[0:20,3]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1.])

In [114]:
print((y[:,3]==1).sum())

21406


In [ ]:
#Test d'implémenter une méthode de ML sur le premier fichier

In [123]:
w_, loss = least_squares(y[:,1], X)

In [130]:

w_dist, loss_dist = least_squares(y[:,0], X)
print("Loss for the distance : {loss} \n".format(loss = loss_dist))


w_E, loss_E = least_squares(y[:,1], X)
print("Loss for the delta_E : {loss} \n".format(loss = loss_E))

w_cos, loss_cos = least_squares(y[:,2], X)
print("Loss for the cos_teta : {loss} \n".format(loss = loss_cos))

w_Q, loss_Q = least_squares(y[:,3], X)
print("Loss for the Q : {loss} \n".format(loss = loss_Q))

Loss for the distance : 0.01800725770361086 

Loss for the delta_E : 3.244195991788376e-27 

Loss for the cos_teta : 0.05344448158955016 

Loss for the Q : 0.02568517364891977 

